In [17]:
import imports
import fea, split, weights, model, train

In [18]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from Bio import SeqIO 

fasta_file = 'input/main/rna_10.fasta'  
sequences = []
metadata = []

for record in SeqIO.parse(fasta_file, "fasta"):
    metadata.append(record.description)  
    seq = str(record.seq).upper()
    sequences.append(seq)
labels = np.ones(len(sequences))
print(metadata[0])
print(labels)
print(sequences[0])

ENST00000657990.1|ENSG00000255248.9|OTTHUMG00000166035.32|OTTHUMT00000510553.1|MIR100HG-255|MIR100HG|972|
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
AGGAGACCAAACACTAGTGTCTCCCCTGCTTCAGGGGACACCCACTGTCAGAGTGAACGACAAAGACGCGCGTGGATGATTTGAAGATATTAGCCTAACAAGATGGAAGCAGGTTTAAAATTTTACATAAGAAGTTACCTTTTTCAGGTAATGGTGTTTTTCTGTAATTTTCCCAAACCTTTCGCTTCCTCAAGTATTGCAAATTTTTAGGGCATGCTTGAATGATTTAATTATGAAGAATAACTACATTTTAAACTTAATTTTTTAAAAAAGAAACCAGTTTTATAATTTTCCTAGCCAAGGAGTAGAGCAGGCATGACATTCAACTCTATGGAGTTTGGAAAATTTTTAGAAATTTATTACTGAACAGCACTTGGTTTTTTGGATTTGAAAATTAAGCCATAGAGCTACAACTAAATTCTGTTTTGCCAAACTTCACAGCACCAGCAGGCTCTGAGCAATCATAAGCATTTGAAACTAGCAGGCACTGCCCCTTAACTCTGTCGCTATCTTGGGCATTTGAATGAAAAAATATAAGTATTACAGGATTGGCTCTAAAAAGCCTACCATGTGTTCTGAATTCCTGGAAGATAGAGCTTATATAAATTACATAATAACAATATTGATTTTTTTTTTTACTTTAGTCATAGATTGGCAATACTATATGGCACTTTATGAAACTATTCCCATGCATATATTCAGTGAGTAGGATTTGATCTTAGCTTTCATGATACAGGCAGTTTCCTTTTAGTGTCATTGAAGGGCTTTATGCCACCTAAAAATGTTTAATAACATAAATTATATATTTGCTTTATAAGTTACATTACTCGGCCTTTGTCACTGCTTATCAGCATTAACCCAT

In [19]:
merscore1, merscore2, merscore3, merscore4, merscore5, merscore6,kmer1, kmer2, kmer3 = fea.generate_features(sequences)

merscore1
merscore2
merscore3
merscore4
merscore5
merscore6


In [20]:
import numpy as np
feat = np.concatenate(
            (
                merscore1, merscore2, merscore3,
                merscore4, merscore5, merscore6,
                kmer1, kmer2, kmer3
            ),
            axis=1
        )
print(feat)

[[ 3.75237020e-02  8.12532926e-02  1.42899625e-01  1.52258560e-01
   1.84686763e-01  2.00142022e-01  3.22016461e-01  1.82098765e-01
   1.63580247e-01  3.32304527e-01  8.33333333e-02  5.24691358e-02
   6.48148148e-02  9.05349794e-02  6.89300412e-02  3.39506173e-02
   8.23045267e-03  6.27572016e-02  5.34979424e-02  4.52674897e-02
   2.88065844e-02  3.18930041e-02  8.53909465e-02  4.21810700e-02
   5.76131687e-02  9.97942387e-02  2.67489712e-02  2.05761317e-02
   2.16049383e-02  2.77777778e-02  1.74897119e-02  1.02880658e-02
   3.08641975e-03  2.05761317e-02  1.64609053e-02  1.95473251e-02
   1.33744856e-02  1.44032922e-02  2.36625514e-02  4.11522634e-03
   1.74897119e-02  4.11522634e-02  1.85185185e-02  1.23456790e-02
   1.64609053e-02  2.16049383e-02  1.54320988e-02  8.23045267e-03
   0.00000000e+00  1.54320988e-02  1.02880658e-03  3.08641975e-03
   2.05761317e-03  1.02880658e-03  1.54320988e-02  1.02880658e-02
   1.23456790e-02  2.46913580e-02  2.26337449e-02  5.14403292e-03
   1.02880

In [21]:
print(feat.shape)
print(labels.shape)

(10, 90)
(10,)


In [22]:
Xt = np.expand_dims(feat, axis=2).astype(np.float32)
yt = labels.astype(np.float32)

print(Xt.shape)  # (10, 90, 1)
print(yt.shape)  # (10,)

(10, 90, 1)
(10,)


In [23]:
from tensorflow.keras.models import load_model
model = load_model("output/rna.keras") 

In [24]:
print(Xt.shape)
predictions = model.predict(Xt)  # (samples, timesteps, features)
predictions = predictions.flatten()  

(10, 90, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step


In [25]:
print(predictions)
print(predictions.shape) 

[0.39052805 0.89840335 0.9998282  0.9749076  0.99813837 0.99732554
 0.53726864 0.350752   0.9997645  0.9998141 ]
(10,)


In [26]:
import pandas as pd

df = pd.DataFrame({
    "metadata": metadata,
    "prediction": predictions
})

In [27]:
df.to_csv("output/rna_10.csv", index=False)
print("Predictions saved to rna_10.csv")

Predictions saved to rna_10.csv
